In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

import cv2
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# Load data
train = pd.read_json("data/iceberg/train.json")
test = pd.read_json("data/iceberg/test.json")

In [3]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])

X_train = np.concatenate([x_band1[:, :, :, np.newaxis],
                          x_band2[:, :, :, np.newaxis],
                          ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)

target_train=train['is_iceberg']

del train

In [4]:
# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])

X_test = np.concatenate([x_band1[:, :, :, np.newaxis],
                         x_band2[:, :, :, np.newaxis],
                         ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
id_test = test['id'].values

del test; del x_band1; del x_band2

In [19]:
# Define CNN Model Architecture (Kaggle can't access the weights file)
img_height = 224
img_width = 224
img_channels = 3
img_dim = (img_height, img_width, img_channels)

def inceptionv3(img_dim=img_dim):
    input_tensor = Input(shape=img_dim)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=img_dim)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = GlobalAveragePooling2D()(x)
   # x = Dense(512, activation='relu')(x)
   # x = Dropout(0.3)(x)
    #x = Dense(512, activation='relu')(x)
    #x = Dropout(0.3)(x)
   # x = Dense(256, activation='relu')(x)
   # x = Dropout(0.3)(x)
    #x = Dense(128, activation='relu')(x)
    #x = Dropout(0.2)(x)
   # x = Dense(128, activation='relu')(x)
   # x = Dropout(0.2)(x)
   # x = Dense(256, activation='relu')(x)
   # x = Dropout(0.1)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs = input_tensor, outputs = predictions)
    
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    #for layer in base_model.layers:
         #layer.trainable = False
    
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics = ['accuracy'])

    
    return model

model = inceptionv3()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
batch_normalization_475 (Bat (None, 224, 224, 3)       12        
_________________________________________________________________
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 2049      
Total params: 21,804,845
Trainable params: 2,055
Non-trainable params: 21,802,790
_________________________________________________________________


#### Change the image size to fit inceptionV3

In [6]:
import scipy
new_shape = (224,224,3)
X_train_new = np.empty(shape=(X_train.shape[0],)+new_shape)
for idx in range(X_train.shape[0]):
    X_train_new[idx] = scipy.misc.imresize(X_train[idx], new_shape)

In [7]:
import scipy
new_shape = (224,224,3)
X_test_new = np.empty(shape=(X_test.shape[0],)+new_shape)
for idx in range(X_test.shape[0]):
    X_test_new[idx] = scipy.misc.imresize(X_test[idx], new_shape)
    
X_test_new /= 255

In [8]:
# Train Model and predict
def train_model(model, batch_size, epochs, img_size, x, y, test, n_fold, kf):
        
    train_scores = []; valid_scores = []
    preds_test = 0  

    i = 1
    for train_index, test_index in kf.split(x):
        x_train = x[train_index]; x_valid = x[test_index]
        y_train = y[train_index]; y_valid = y[test_index]


        
        #batch_size=64
        # Define the image transformations here
        train_gen = ImageDataGenerator(horizontal_flip = True,
                                 vertical_flip = True,
                                 width_shift_range = 0.,
                                 height_shift_range = 0.,
                                 channel_shift_range=0,
                                 zoom_range = 0.2,
                                 rotation_range = 10,
                                 rescale=1./255,
                                 )
        valid_gen = ImageDataGenerator(horizontal_flip = True,
                                 vertical_flip = True,
                                 width_shift_range = 0.,
                                 height_shift_range = 0.,
                                 channel_shift_range=0,
                                 zoom_range = 0.2,
                                 rotation_range = 10,
                                 rescale=1./255,
                                )
        #test_gen = ImageDataGenerator(rescale=1./255)
        
        callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, cooldown=1, 
                               verbose=1, min_lr=1e-7),
             ModelCheckpoint(filepath='inception.fold_' + str(i) + '.hdf5', verbose=1,
                             save_best_only=True, save_weights_only=True, mode='auto')]

        train_steps = len(x_train) / batch_size
        valid_steps = len(x_valid) / batch_size
          
        model = model

        model.fit_generator(train_gen.flow(x_train, y_train, batch_size=32),
                            steps_per_epoch=len(x_train) / 32, epochs=epochs, 
                            callbacks=callbacks, validation_data= valid_gen.flow(x_valid, y_valid, batch_size=32),
                           validation_steps = valid_steps)

        model.load_weights(filepath='inception.fold_' + str(i) + '.hdf5')

        
        print('----------------------------------------')
        print('Running train evaluation on fold {}'.format(i))
        train_score = model.evaluate_generator(train_gen.flow(x_valid, y_valid, batch_size=32), steps=train_steps)        
        print('Running validation evaluation on fold {}'.format(i))
        valid_score = model.evaluate_generator(valid_gen.flow(x_valid, y_valid, batch_size=32), steps=valid_steps)
        print('----------------------------------------')   
        
        print('Train loss: {:0.5f}\n Train acc: {:0.5f} for fold {}'.format(train_score[0],
                                                                            train_score[1], i))
        print('Valid loss: {:0.5f}\n Valid acc: {:0.5f} for fold {}'.format(valid_score[0],
                                                                            valid_score[1], i))
        print('----------------------------------------')

        train_scores.append(train_score[1])
        valid_scores.append(valid_score[1])
        print('Avg Train Acc: {:0.5f}\nAvg Valid Acc: {:0.5f} after {} folds'.format
              (np.mean(train_scores), np.mean(valid_scores), i))
        print('----------------------------------------')
        
        #Getting test scores
        print('Running test predictions with fold {}'.format(i)) 
        temp_test = model.predict(test,batch_size = 32, verbose=1)
        preds_test +=temp_test.reshape(temp_test.shape[0])
        

        print('\n\n')

        i += 1

        if i <= n_fold:
            print('Now beginning training for fold {}\n\n'.format(i))
        else:
            print('Finished training!')

    preds_test /= n_fold
    
    
    return preds_test

In [20]:
batch_size = 32
epochs = 50
n_fold = 3
img_size = (img_height, img_width)
kf = KFold(n_splits=n_fold, shuffle=True)

prediction = train_model(model, batch_size, epochs, img_size, X_train_new, 
                                target_train, X_test_new, n_fold, kf)



Epoch 1/50
34/33 [==============================] - 39s - loss: 0.7104 - acc: 0.5153 - val_loss: 0.7042 - val_acc: 0.5084
Epoch 2/50
23/33 [===================>..........] - ETA: 6s - loss: 0.7009 - acc: 0.5380

KeyboardInterrupt: 

In [12]:
submit = pd.DataFrame({'id': id_test, 'is_iceberg': prediction.reshape((prediction.shape[0]))})
#submit.to_csv('./submission_inception.csv', index=False)
submit.head(10)

,id,is_iceberg
0,5941774d,0.464490
1,4023181e,0.489352
2,b20200e4,0.516133
3,e7f018bb,0.539427
4,4371c8c3,0.692291
5,a8d9b1fd,0.219566
6,29e7727e,0.467108
7,92a51ffb,0.543889
8,c769ac97,0.372378
9,aee0547d,0.369774
